# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/train.csv')

train.describe().apply(lambda s: s.apply('{0:.5f}'.format))

test=pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/test.csv')
test.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,...,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000
mean,10.65874,-1.62424,10.70745,6.78821,11.07640,-5.05056,5.41516,16.52914,0.27713,7.56941,...,3.18977,7.45827,1.92594,3.32202,17.99697,-0.13366,2.29090,8.91243,15.86918,-3.24634
std,3.03672,4.04051,2.63389,2.05272,1.61646,7.86929,0.86469,3.42448,3.33337,1.23187,...,4.55124,3.02519,1.47997,3.99560,3.14065,1.42968,5.44635,0.92090,3.00872,10.39859
min,0.18870,-15.04340,2.35520,-0.02240,5.48440,-27.76700,2.21640,5.71370,-9.95600,4.24330,...,-14.09330,-2.40700,-3.34090,-11.41310,9.38280,-4.91190,-13.94420,6.16960,6.58400,-39.45780
25%,8.44298,-4.70012,8.73560,5.23050,9.89108,-11.20140,4.77260,13.93390,-2.30390,6.62380,...,-0.09500,5.16650,0.88298,0.58760,15.63478,-1.16070,-1.94860,8.26008,13.84727,-11.12400
50%,10.51380,-1.59050,10.56070,6.82235,11.09975,-4.83410,5.39160,16.42270,0.37200,7.63200,...,3.16240,7.37900,1.89260,3.42850,17.97760,-0.16200,2.40360,8.89280,15.94340,-2.72595
75%,12.73960,1.34340,12.49503,8.32760,12.25340,0.94257,6.00580,19.09455,2.93003,8.58482,...,6.33648,9.53110,2.95600,6.17420,20.39173,0.83790,6.51980,9.59590,18.04520,4.93540
max,22.32340,9.38510,18.71410,13.14200,16.03710,17.25370,8.30250,28.29280,9.66550,11.00360,...,20.35900,16.71650,8.00500,17.63260,27.94780,4.54540,15.92070,12.27580,26.53840,27.90740


# Split and normalize data

In [5]:
#shuffle data
train = train.sample(frac=1)

X = train.drop(['target', 'ID_code'], axis=1)
y=train[['target']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X = scaler.fit_transform(X) 

from sklearn.model_selection import train_test_split
# split into 80% train, 10% validation, 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=1) # 0.2 x 0.5 = 0.1

# Train and evaluate model

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [30]:
l2_reg = keras.regularizers.l2(0.1)

input_ = keras.layers.Input(shape=[200])
hidden1 = keras.layers.Dense(200, activation="relu", kernel_constraint=keras.constraints.max_norm(1.0), kernel_regularizer=l2_reg)(input_)
hidden1 = tf.keras.layers.Dropout(0.6)(hidden1)
hidden2 = keras.layers.Dense(200, activation="relu", kernel_constraint=keras.constraints.max_norm(1.0), kernel_regularizer=l2_reg)(hidden1)
hidden2 = tf.keras.layers.Dropout(0.6)(hidden2)
hidden3 = keras.layers.Dense(200, activation="relu", kernel_constraint=keras.constraints.max_norm(1.0), kernel_regularizer=l2_reg)(hidden2)
hidden3 = tf.keras.layers.Dropout(0.6)(hidden3)
concat = keras.layers.Concatenate()([input_, hidden3])
output = keras.layers.Dense(1, activation="sigmoid")(concat)
model = keras.Model(inputs=[input_], outputs=[output])

#checkpoint_cb = keras.callbacks.ModelCheckpoint("m1.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard('/content/drive/MyDrive/Data Science Project - Team D/data/logs')

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


model.compile(loss="BinaryCrossentropy", optimizer="adam", metrics=METRICS)
history = model.fit(X_train, y_train, epochs=100, batch_size=1000, validation_data=(X_valid, y_valid), callbacks=[early_stopping_cb])

Epoch 1/100
160/160 [==============================] - 4s 14ms/step - loss: 13.2555 - tp: 1330.0000 - fp: 4177.0000 - tn: 139701.0000 - fn: 14792.0000 - accuracy: 0.8814 - precision: 0.2415 - recall: 0.0825 - auc: 0.6892 - prc: 0.1863 - val_loss: 0.4923 - val_tp: 112.0000 - val_fp: 31.0000 - val_tn: 18001.0000 - val_fn: 1856.0000 - val_accuracy: 0.9057 - val_precision: 0.7832 - val_recall: 0.0569 - val_auc: 0.8337 - val_prc: 0.4342
Epoch 2/100
160/160 [==============================] - 1s 8ms/step - loss: 0.2820 - tp: 2128.0000 - fp: 686.0000 - tn: 143192.0000 - fn: 13994.0000 - accuracy: 0.9082 - precision: 0.7562 - recall: 0.1320 - auc: 0.8386 - prc: 0.4589 - val_loss: 0.2331 - val_tp: 363.0000 - val_fp: 109.0000 - val_tn: 17923.0000 - val_fn: 1605.0000 - val_accuracy: 0.9143 - val_precision: 0.7691 - val_recall: 0.1845 - val_auc: 0.8595 - val_prc: 0.4966
Epoch 3/100
160/160 [==============================] - 1s 8ms/step - loss: 0.2388 - tp: 3443.0000 - fp: 1443.0000 - tn: 142435.000

In [31]:
sub=pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/sample_submission.csv')
sub.head()

X_test = test.drop(['ID_code'], axis=1)

test.head()

y_pred = model.predict(X_test)

sub["target"] = y_pred
sub.describe()

sub.to_csv('/content/drive/MyDrive/Data Science Project - Team D/submission/various_submissions/fe0_ann.csv', index=False)

6250/6250 [==============================] - 10s 2ms/step


# Sources:
https://www.amazon.de/-/en/Aurélien-Géron/dp/1098125975/ref=sr_1_1?keywords=hands-on+machine+learning+with+scikit-learn%2C+keras%2C+and+tensorflow&qid=1671798944&sprefix=hands-on+mach%2Caps%2C167&sr=8-1